# Custom JS functions
* Import libraries
* define parameter array
* define widgets
* define functions computed on the client
* define histogram array
* define figures
* draw data
* possible optimization by using v_func instead of func

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.CDSAlias import CDSAlias
from RootInteractive.InteractiveDrawing.bokeh.CustomJSNAryFunction import CustomJSNAryFunction
from RootInteractive.InteractiveDrawing.bokeh.DownsamplerCDS import DownsamplerCDS

from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import bokehDrawSA

from bokeh.models.sources import ColumnDataSource
from bokeh.models.widgets import Slider
from bokeh.models.layouts import Column

from bokeh.plotting import Figure, output_file

import pandas as pd
import numpy as np

output_notebook()

* Generate random data

In [ ]:
df = pd.DataFrame(np.random.random_sample(size=(20000, 6)), columns=list('ABCDEF'))

* Create parameters for visualization and widgets controlling them

In [ ]:
parameterArray = [
    # Parameters for visualization
    {"name": "colorZ", "value":"A", "options":["A", "B"]},
    {"name": "size", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    # Parameters for custom JS columns
    {"name": "paramX", "value":10, "range": [-20, 20]},
    {"name": "C_cut", "value": 1, "range": [0, 1]}
]
widgetParams=[
    ['range', ['A']],
    ['range', ['B', 0, 1, 0.1, 0, 1]],

    ['range', ['C'], {'type': 'minmax'}],
    ['range', ['D'], {'type': 'sigma', 'bins': 10, 'sigma': 3}],
    ['range', ['E'], {'type': 'sigmaMed', 'bins': 10, 'sigma': 3}],
    #['slider','F', ['@min()','@max()','@med','@min()','@median()+3*#tlm()']], # to be implmneted
    ['select',["colorZ"], {"callback": "parameter", "default": 0}],
    ['slider',["size"], {"callback": "parameter"}],
    ['select',["legendFontSize"], {"callback": "parameter", "default": 2}],
    ['slider',["C_cut"], {"callback": "parameter"}],
    ['slider',["paramX"], {"callback": "parameter"}],
]

widgetLayoutDesc={
    "Selection": [[0, 1, 2], [3, 4], {'sizing_mode': 'scale_width'}],
    "Graphics": [[5, 6, 7], {'sizing_mode': 'scale_width'}],
    "CustomJS functions": [[8, 9]]
    }

* Define custom JS columns
* Parameters each element of the array takes
    * `name` - the name used
    * `variables` - the names of the columns used by the transform
    * `parameters` - an unfortunate name - the uniform parameters controlled by the widgets
    * `expr`, `func`, `v_func` - only one can be used, define the function that will be computed
        * `expr` - the python expression that will be computed either on the client or the server - in this case, parameters are detected automatically, however, only elementwise operations are supported right now
        * `func`- the function to be computed on the client - can be a parameter in parameterArray editable by text
        * `v_func` - the function to be computed on the client, with vectors on input and output
* Shorthand: `(name, expr)` or `(name, expr, table)` tuple

In [ ]:
aliasArray = [
    # Custom JS columns can be directly shown
    {
        "name": "A_mul_paramX_plus_B",
        "variables": ["A", "B"],
        "parameters": ["paramX"],
        "func": "return paramX * A + B" 
    },
    # They can also be used as histogram weights
    {
        "name": "C_accepted",
        "expr": "C < C_cut"
    },
    # By specifying the context parameter, custom JS columns can be made in histograms too
    {
        "name": "effC",
        "variables": ["bin_count", "entries_C_cut"],
        "func": "return entries_C_cut / bin_count",
        "context": "histoA"
    },
    # Shorthand notation
    ("effC", "entries_C_cut / bin_count", "histoAC"),
    # Custom function editable by a textedit
    {
        "name": "custom_column",
        "variables": ["A", "B", "C", "A_mul_paramX_plus_B"],
        "func": "CustomFunc"
    }
]

* Create histograms
* new parameter in histoArray - `histograms` - create multiple histograms with the same binning within one CDS
* takes a dictionary
    * `weights` - the weightws to use for the histogran
    * `cumulative` - whether the histogram should be cumulative - applies only to 1D histograms
    * `density` - whether the count should be divided by bin width - only implemented for 1D histograms

In [ ]:
histoArray = [
    {
        "name": "histoA", "variables": ["A"], "nbins": 10, "histograms": {
            "entries_C_cut": {
                "weights": "C_accepted"
            }
        }
    },
    {
        "name": "histoAC", "variables": ["A", "C"], "nbins": [6, 6], 
        "histograms": {
            "entries_C_cut": {
                "weights": "C_accepted"
            }
        }
    }
]

In [ ]:
figureArray = [
    # Directly draw the column
    [['A'], ['B', '4*A+B', 'A_mul_paramX_plus_B']],
    # Multiple weights columns can be specified in the same histogram CDS using the new histograms parameter
    [['bin_center'], ['bin_count', 'entries_C_cut'], {"source":"histoA"}],
    # Custom JS functions can be used on histograms to transform the computed histogram values
    [['bin_center'], ['effC'], {"source":"histoA"}],
    # This works for both 1D and ND histograms
    [['bin_center_0'], ['effC'], {"colorZvar": "bin_center_1", "source":"histoAC"}],
    {"size":"size", "legend_options": {"label_text_font_size": "legendFontSize"}}
]
figureLayoutDesc=[
        [0, 1, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        [2, 3, {'plot_height': 200, 'sizing_mode': 'scale_width'}]
        ]
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]

In [ ]:
output_file("test_AliasBokehDraw.html")
bokehDrawSA.fromArray(df, None, figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, parameterArray=parameterArray,
                          widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=300,
                           aliasArray=aliasArray, histogramArray=histoArray)